In [42]:
from pymongo import MongoClient
import pymongo
import datetime
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm

In [43]:
client = MongoClient(host='114.70.235.37', port=27017)
print(client.list_database_names())

['admin', 'config', 'local', 'mydb']


In [84]:
db = client['mydb'].xy
result= db.find().sort([("date", pymongo.DESCENDING)]).limit(2)

data = list(result)[0]
start_date = data["date"] - datetime.timedelta(seconds=5)
seq = list(db.find({"date": {'$gte' : start_date, '$lt' : data["date"]}}))

In [85]:
list_data = list()
for j in seq:
    del j['date']
    del j['_id']
    
    list_data.append(list(j.values()))

In [86]:
k = int(len(list_data)/30)
list_2 = []
for i in range(1,31):
    list_2.append(list_data[k*i])

In [87]:
list_data = np.array(list_2, dtype=np.float32)
list_data = np.expand_dims(list_data, axis=0)

In [88]:
y_false = np.array([0 for w in range(len(list_data))])
print(y_false)
print(list_data.shape)
print(y_false.shape)

[0]
(1, 30, 20)
(1,)


In [89]:
# dataloaders
batch_size = 32
device='cuda'

class SentimentLSTM(nn.Module):

    def __init__(self, seq_size, output_size, hidden_dim, n_layers):
        
        super(SentimentLSTM, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # embedding and LSTM layers
        self.lstm = nn.LSTM(seq_size, hidden_dim, n_layers, batch_first=True, dropout=0.2)
        
        # linear and sigmoid layer
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        batch_size = x.size(0)
        
        lstm_out, hidden = self.lstm(x)
        ## lstm_out.shape [batch, seq_len, hidden_dim]
        ## hidden [batch, hidden_dim]

        # fully connected layer        
        out = self.fc(lstm_out[:, -1, :])
        ## out.shape: [n_layer * n_direction, batch, output_size]

        # sigmoid function
        sig_out = self.sig(out)
        
        # return last sigmoid output and hidden state
        return sig_out, out

In [90]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        # nn.init.uniform_(param.data, -0.1, 0.1)

In [91]:
hidden_dim = 512
n_layers = 4
lr = 0.01
output_size = 1

model = SentimentLSTM(20, output_size, hidden_dim, n_layers)
model.to('cuda')

SentimentLSTM(
  (lstm): LSTM(20, 512, num_layers=4, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [92]:
model.load_state_dict(torch.load("model_weight.pth"))

<All keys matched successfully>

In [93]:
def predict(model, test_loader, sequence_length=30):
    
    model.eval()
    
    batch_size = 32
    acc_list = list()
    
    for inputs, labels in test_loader:
        inputs, labels = inputs.cuda(), labels.cuda()
        
        output, h = model(inputs)
        
        pred = torch.round(output.squeeze())
        
        accuracy = (pred==labels).sum().item() / 32
        acc_list.append(accuracy)

        #print('Prediction value, pre-rounding: ', output.item())

        # print custom response based on whether test_review is pos/neg
        
    print(np.mean(acc_list))

In [94]:
test_data = torch.load('/project/annotation/test_v2.pt')
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)
tmp_data = TensorDataset(torch.from_numpy(list_data), torch.from_numpy(y_false))
tmp_loader = DataLoader(tmp_data, shuffle=False, batch_size=batch_size)



In [95]:
predict(model, tmp_loader)

0.03125


In [96]:
tmp_data[0]

(tensor([[ 3.1322e-01, -3.1059e+01, -1.8854e+02, -1.4704e+02,  2.1160e+02,
          -1.4212e+02,  4.4922e+02,  3.4480e+02,  4.9078e+01,  3.3988e+02,
           2.6068e+02,  1.9776e+02,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 1.8242e+00, -3.1912e+01, -1.8679e+02, -1.4732e+02,  2.1102e+02,
          -1.4065e+02,  4.4860e+02,  3.4591e+02,  5.0787e+01,  3.3924e+02,
           2.6181e+02,  1.9859e+02,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 2.6270e+00, -3.1728e+01, -1.8643e+02, -1.4756e+02,  2.1111e+02,
          -1.3972e+02,  4.4892e+02,  3.4659e+02,  5.1374e+01,  3.3875e+02,
           2.6249e+02,  1.9903e+02,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 2.7519e+00, -3.1083e+01, -1.8550e+02, -1.4611e+02,  2.1271e+02,
          -1.3869e+02,